In [ ]:
### ANTES DE EXECUTAR GARANTIR QUE O BANCO NEO4J ESTÁ NO AR ###
### ALTERAR A VARIAVEL DATA #######

import seaborn as sns
import pandas as pd
from neo4j import GraphDatabase
import hashlib
from dotenv import load_dotenv
import os
from datetime import datetime


load_dotenv()
# Data da Consulta
date = "07/02/2025"

# Converte para um objeto datetime
date_obj = datetime.strptime(date, "%d/%m/%Y")

# Converte para o formato yyyy-mm-dd
formatted_date = date_obj.strftime("%Y-%m-%d")

# Comando Python que você deseja executar
cmd_special = "python showusage.py -ds " + formatted_date  + " -report SPECIAL -t ALMEIDA -csv"

cmd_compartment = "python showusage.py -ds " + formatted_date  + "  -report COMPARTMENT -t ALMEIDA -csv"


os.system(cmd_special)

os.system(cmd_compartment)


def computeMD5hash(my_string):
    m = hashlib.md5()
    m.update(my_string.encode('utf-8'))
    return m.hexdigest()

# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
URI =os.getenv('NEO4J_URI')
AUTH = (os.getenv('NEO4J_USER'), os.getenv('NEO4J_PASSWORD'))

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()


driver = GraphDatabase.driver(URI, auth=AUTH)
session = driver.session(database="neo4j")

file_compartment = "usage_compartments.csv"
file_special = "usage_special.csv"
file_name = "dados.csv"
df_compartment = pd.read_csv(file_compartment)

df_special = pd.read_csv(file_special)

df_compartment = df_compartment[["Compartment Path", "Service", "Currency", "Cost"]] # as duas primeiras colunas do dataframe

df_special = df_special.iloc[:, 0:4] # as quatro primeiras colunas do dataframe

df_compartment.set_index('Service')
df_special.set_index('Service')

df = df_special.merge(df_compartment, on='Service', how='left')


query = '''
unwind $data as row
MERGE (c:Compartment {name: row.Compartment_Path, regiao: row.Region})
MERGE (p:Product {sku: row.Product_SKU, name: row.Product_Name, servico: row.Service})
MERGE (c)-[:CONSUMIU {cost: row.Cost, date:row.Date}]->(p)

'''

df.to_csv(file_name, sep=';', encoding='utf-8', index=False, header=True)

for key, value in df.iterrows():
    data = {
        'Service': value['Service'],
        'Region': value['Region'],
        'Product_SKU': value['Product SKU'],
        'Product_Name': value['Product Name'],
        'Compartment_Path': value['Compartment Path'],
        'Currency': value['Currency'],
        'Cost': value['Cost'],
        'Cost_id': computeMD5hash(value['Product SKU'] + value['Compartment Path']),
        'Date': date
    }
    records, summary, keys = driver.execute_query(query, data=data, )
    print("The query `{query}` returned {records_count} records in {time} ms.".format(query=summary.query, records_count=len(records),time=summary.result_available_after, ))



# session/driver usage

session.close()
driver.close()